In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

print(tf.test.is_gpu_available())

train_df = pd.read_csv('task3_2021.training.csv')
test_df = pd.read_csv('task3_2021.test.csv')
n_pre_train_y = train_df['target']
n_pre_train_x = train_df.drop(['target'], axis=1)
n_pre_test_x = test_df
# display(n_pre_train_x.head(), n_pre_train_y.head(), n_pre_train_y.nunique(), n_pre_test_x.head())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2021-12-21 03:06:10.074229: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-21 03:06:11.156848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 03:06:11.157336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 03:06:11.161435: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

True
time: 16.6 s (started: 2021-12-21 03:05:58 +00:00)


In [2]:
def oh_pre(x):
    for k in ['f15', 'f16', 'f17']:
        tmp_one_hot_k =  pd.get_dummies(x[k], prefix=f"{k}")
        x = pd.concat([x, tmp_one_hot_k],axis=1)
        x = x.drop([k], axis=1)
    return x

oh_pre_train_x = oh_pre(n_pre_train_x)
oh_pre_test_x = oh_pre(n_pre_test_x)
scaler = StandardScaler()
train_x = scaler.fit_transform(oh_pre_train_x)
test_x = scaler.transform(oh_pre_test_x)

ohenc = OneHotEncoder(handle_unknown="ignore", sparse=False)
train_y = ohenc.fit_transform(n_pre_train_y.values.reshape(-1,1))
# mse = make_scorer(mean_squared_error, squared=False)nunique

time: 126 ms (started: 2021-12-21 03:06:15 +00:00)


In [36]:
from tensorflow.keras import regularizers

class DNN(tf.keras.Model):
  def __init__(self):
    super(DNN,self).__init__()
    # self.dense1 = tf.keras.layers.Dense(24, activation=tf.nn.relu,
                                        # kernel_regularizer=regularizers.l1_l2(l1=1e-7, l2=1e-7),
                                        # bias_regularizer=regularizers.l2(1e-7),)
    self.dense1 = tf.keras.layers.Dense(16, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    # x = self.dense3(x)
    return self.output_layer(x)

def scheduler(epoch, lr):
    if epoch < 100:
        return lr
    else:
        return lr*tf.math.exp(-1e-2)


callback_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=100,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

batch_size = 50
epochs = 5000
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model = DNN()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback_early], verbose=1)
model.summary()

Epoch 1/5000
180/180 [==============================] - 0s 1ms/step - loss: 2.2508 - accuracy: 0.1523 - val_loss: 2.0585 - val_accuracy: 0.2120
Epoch 2/5000
180/180 [==============================] - 0s 918us/step - loss: 1.8423 - accuracy: 0.2764 - val_loss: 1.6360 - val_accuracy: 0.3390
Epoch 3/5000
180/180 [==============================] - 0s 918us/step - loss: 1.4547 - accuracy: 0.4071 - val_loss: 1.3031 - val_accuracy: 0.4620
Epoch 4/5000
180/180 [==============================] - 0s 904us/step - loss: 1.1683 - accuracy: 0.5481 - val_loss: 1.0642 - val_accuracy: 0.5980
Epoch 5/5000
180/180 [==============================] - 0s 923us/step - loss: 0.9627 - accuracy: 0.6586 - val_loss: 0.8832 - val_accuracy: 0.6710
Epoch 6/5000
180/180 [==============================] - 0s 916us/step - loss: 0.8133 - accuracy: 0.7289 - val_loss: 0.7567 - val_accuracy: 0.7450
Epoch 7/5000
180/180 [==============================] - 0s 895us/step - loss: 0.7010 - accuracy: 0.7751 - val_loss: 0.6582 - v

In [30]:
from pandas import RangeIndex

nn_pred = model.predict(test_x)
for i in range(,0,-1):
    thres = i/10
    pred = np.where(nn_pred<thres, 0, 1)
    output = ohenc.inverse_transform(pred)
    check = (output==None).sum()
    print(f"threshold={thres} => Number of None is {check}")
    if check==0:break

if check==0:
    pd.Series(output.reshape(-1), index=RangeIndex(1, 2001), name='Predicted').to_csv('ans_nn_4.csv',index_label='id')

threshold=0.9 => Number of None is 738
threshold=0.8 => Number of None is 479
threshold=0.7 => Number of None is 306
threshold=0.6 => Number of None is 160
threshold=0.5 => Number of None is 2
threshold=0.4 => Number of None is 0
time: 72.4 ms (started: 2021-12-21 03:54:42 +00:00)
